In [1]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [3]:
df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [4]:
print('열의 갯수 : ', len(df.columns))
print(df.columns)

열의 갯수 :  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [5]:
df['headline'].isnull().sum()

0

In [11]:
headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [14]:
print('총 샘플의 수 : ', len(headline))

총 샘플의 수 :  1324


In [15]:
headline = [n for n in headline if n != "Unknown"]
print('노이즈 값 제거 후 샘플의 갯수 : ', len(headline))

노이즈 값 제거 후 샘플의 갯수 :  1214


In [16]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [19]:
def repreprocessing(s):
    s = s.encode("utf8").decode("ascii", 'ignore') # 유니코드를 아스키코드로 변환
    return ''.join(c for c in s if c not in punctuation).lower() # punctuation : 구두점 제거
    # join : 리스트에서 문자열로 변환
text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [20]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [23]:
sequences = []
for line in text:
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [24]:
index_to_word={}
for key, value in t.word_index.items():
    index_to_word[value] = key
print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [25]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


In [26]:
sequences = pad_sequences(sequences, maxlen=max_len)
print(sequences)

[[   0    0    0 ...    0   99  269]
 [   0    0    0 ...   99  269  371]
 [   0    0    0 ...  269  371 1115]
 ...
 [   0    0    0 ...    8 3493  115]
 [   0    0    0 ... 3493  115    2]
 [   0    0    0 ...  115    2 1025]]


In [27]:
sequecnces = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [28]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [29]:
y = to_categorical(y, vocab_size)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [31]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len - 1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 2s - loss: 7.6300 - accuracy: 0.0259
Epoch 2/200
244/244 - 2s - loss: 7.1096 - accuracy: 0.0299
Epoch 3/200
244/244 - 2s - loss: 6.9724 - accuracy: 0.0361
Epoch 4/200
244/244 - 2s - loss: 6.8442 - accuracy: 0.0368
Epoch 5/200
244/244 - 2s - loss: 6.6941 - accuracy: 0.0437
Epoch 6/200
244/244 - 2s - loss: 6.5247 - accuracy: 0.0470
Epoch 7/200
244/244 - 2s - loss: 6.3348 - accuracy: 0.0506
Epoch 8/200
244/244 - 2s - loss: 6.1399 - accuracy: 0.0564
Epoch 9/200
244/244 - 2s - loss: 5.9513 - accuracy: 0.0611
Epoch 10/200
244/244 - 2s - loss: 5.7749 - accuracy: 0.0646
Epoch 11/200
244/244 - 2s - loss: 5.6087 - accuracy: 0.0698
Epoch 12/200
244/244 - 2s - loss: 5.4510 - accuracy: 0.0709
Epoch 13/200
244/244 - 2s - loss: 5.3049 - accuracy: 0.0777
Epoch 14/200
244/244 - 2s - loss: 5.1582 - accuracy: 0.0847
Epoch 15/200
244/244 - 2s - loss: 5.0218 - accuracy: 0.0895
Epoch 16/200
244/244 - 2s - loss: 4.8869 - accuracy: 0.1000
Epoch 17/200
244/244 - 2s - loss: 4.7594 - accura

244/244 - 2s - loss: 0.3702 - accuracy: 0.9150
Epoch 138/200
244/244 - 2s - loss: 0.3627 - accuracy: 0.9135
Epoch 139/200
244/244 - 2s - loss: 0.3599 - accuracy: 0.9141
Epoch 140/200
244/244 - 2s - loss: 0.3550 - accuracy: 0.9161
Epoch 141/200
244/244 - 2s - loss: 0.3794 - accuracy: 0.9116
Epoch 142/200
244/244 - 2s - loss: 0.4007 - accuracy: 0.9072
Epoch 143/200
244/244 - 2s - loss: 0.3593 - accuracy: 0.9136
Epoch 144/200
244/244 - 3s - loss: 0.3410 - accuracy: 0.9152
Epoch 145/200
244/244 - 2s - loss: 0.3355 - accuracy: 0.9150
Epoch 146/200
244/244 - 2s - loss: 0.3310 - accuracy: 0.9166
Epoch 147/200
244/244 - 2s - loss: 0.3281 - accuracy: 0.9167
Epoch 148/200
244/244 - 2s - loss: 0.3248 - accuracy: 0.9172
Epoch 149/200
244/244 - 3s - loss: 0.3218 - accuracy: 0.9159
Epoch 150/200
244/244 - 3s - loss: 0.3204 - accuracy: 0.9173
Epoch 151/200
244/244 - 3s - loss: 0.3166 - accuracy: 0.9158
Epoch 152/200
244/244 - 2s - loss: 0.3148 - accuracy: 0.9171
Epoch 153/200
244/244 - 2s - loss: 0.3

In [ ]:
def sentence_generation(model, t, current_word, n): 
    init_word = current_word 
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] 
        encoded = pad_sequences([encoded], maxlen=5) # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)

        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence